In [2]:
import pandas as pd
import time
import urllib.request
import urllib.parse
import json
import hashlib
import base64
import matplotlib.pyplot as plt
import jionlp as jio
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

`jio.help()` is provided to search how to use jio functions.
Or browse `https://github.com/dongrixinyu/JioNLP` to get help.
# jiojio - Failed to load C funcs, use py func instead.


In [3]:
## load data
comment = pd.read_csv('/Users/tunan/Code/Python/reptile/scrapy_learning/carpro/carpro/spiders/id4_comment.csv')

In [4]:
## data exploration

comment = pd.DataFrame(comment)

comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  90 non-null     int64  
 1   name        90 non-null     object 
 2   comment     90 non-null     object 
 3   mark        90 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.9+ KB


In [7]:
comment = comment[['name', 'comment', 'mark']]
comment.head(5)

,name,comment,mark
0,\n 2022款ID.4 X 智享长续航版\n,写完回头写下这篇文字，原来写了这么多。献给一些心动但又想得到中肯建议的人。\n\n先说外观，...,4.14
1,\n 2022款ID.4 X 纯净长续航版\n,外观，内饰都是我能接受的比较中庸，但也是比较排上风格。不激进，不寒碜。\n电动车，首先他是一...,3.86
2,\n 2022款ID.4 X 极智长续航版\n,外观：颜值符合我的审美，家人也都说好看，耐看。原来已经订了宋plus ev的，看到id4x的...,4.07
3,\n 2022款ID.4 X 劲能四驱版\n,外观方面：\n车身线条营造出饱满的样子显车子很大，前脸和尾部的贯穿式车灯效果很棒，特别是晚上...,3.86
4,\n 2022款ID.4 X 智享长续航版\n,首先说一下，下定ID4之前也对比了很多车型，但是最终还是选择上汽大众ID4，3.25下定 ...,4.43


In [9]:
comment.count()

name       90
comment    90
mark       90
dtype: int64

In [20]:
comment.describe()


,mark
count,90.000000
mean,4.033333
std,0.361374
min,2.500000
25%,3.860000
50%,3.930000
75%,4.192500
max,4.930000


In [49]:
# emotion choice api
def emotion(url,x_appid,api_key,text):
    body = urllib.parse.urlencode({'text': text}).encode('utf-8')
    param = {"type": "dependent"}
    x_param = base64.b64encode(json.dumps(param).replace(' ', '').encode('utf-8'))
    x_time = str(int(time.time()))
    x_checksum = hashlib.md5(api_key.encode('utf-8') + str(x_time).encode('utf-8') + x_param).hexdigest()
    x_header = {'X-Appid': x_appid,
                'X-CurTime': x_time,
                'X-Param': x_param,
                'X-CheckSum': x_checksum}
    req = urllib.request.Request(url, body, x_header)
    result = urllib.request.urlopen(req)
    result = result.read()
    # print(result.decode('utf-8'))
    return result


In [ ]:
url = 'http://ltpapi.xfyun.cn/v2/sa'
x_appid = '26938d57'
api_key = '91efbf93df628100dd68f92ddf0e09ec'

mark_list = []
for i in range(len(comment)):
    mark_dic = {}
    text = comment['comment'].iloc[i]
    res = jio.split_sentence(text, criterion='fine')
    for phrase in res:
        phrase.replace('\n', '')
        result = emotion(url,x_appid,api_key,phrase)
        data = json.loads(result.decode('utf-8'))
        if data['code'] == 0:
            print(data['data'])
            # mark_dic.setdefault(i, []).append(data['sentiment'])
    # print(mark_dic)